# 観光消費構造分析 ― 国内移動を含むサービス消費の視点から ―

### 目的とアジェンダ

#### 分析の目的
分析では、訪日観光消費を費目別構成および国籍・地域別の消費水準の観点から整理し、
コロナ禍前と最新年の比較を通じて、訪日観光消費構造がどのように変化しているのかを明らかにすることを目的とする。

特に、宿泊費・交通費・娯楽等サービス費といった国内移動を含むサービス費目に着目し、
これらの消費が特定の国籍に限定された現象なのか、ある程度まとまった国群として観測される構造なのかを検証する。

その上で、買物代との対比を通じて、訪日観光消費における
モノ消費中心の市場とサービス消費中心の市場の違いを整理し、
今後のインバウンド戦略において重視すべき視点に関するインサイトを得ることを目的とする。

#### 主要な分析ステップ

1.  1人あたりの観光消費額の時系列比較による消費単価の変化把握
2.  消費構造の変化  
  2.1 費目別構成比の比較  
  2.2 国籍・地域間の分散構造分析によるコロナ禍前後における消費構造の均質化・分化の確認  
3.  費目間相関分析による交通費・宿泊費・娯楽等サービス費の連動性の確認
4.  国内移動を含むサービス消費構造の一般性に関する検証  
  4.1 サービス費目別消費額水準の国籍・地域別整理（複数費目で高水準となる国籍・地域の把握）  
  4.2 サービス費目上位国の重なりに関する定量的確認（特定国の例外か、国群としての構造かの検証）  
  4.3. 買物代消費額水準との対比（サービス消費構造との違いの確認）

### 環境構築とデータ準備

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os 
from IPython.display import display, Markdown
from itertools import combinations


# --- 定数定義 ---
MAIN_EXPENSE_ITEMS = ['宿泊費', '飲食費', '買物代', '交通費', '娯楽等サービス費', 'その他'] 
HIGH_VALUE_ITEMS = ['宿泊費', '娯楽等サービス費'] 
LATEST_YEAR = 2024
COMPARISON_YEAR = 2019

COUNTRY_ORDER = ['全国籍･地域', '中国', '韓国', '台湾', '香港', '米国', 'オーストラリア', 'その他'] 

DATA_PATH = '../data/'
DATA_FILENAME = 'inbound_spending.csv'

# データ読込
try:
    df = pd.read_csv(os.path.join(DATA_PATH, DATA_FILENAME))
    
except FileNotFoundError as e:
    print(f"エラー: {DATA_FILENAME} が見つかりません。データファイルの配置を確認してください。")
    raise
    
# 期間のフィルタリング
df = df[(df['year'] >= COMPARISON_YEAR) & (df['year'] <= LATEST_YEAR)].copy()

# 'period_Quarter'を使って期間文字列を作成する。
df['period'] = df['year'].astype(str) + '年' + df['period_Quarter']

# --- ★★★ 変更箇所: Quarter_Numericの作成を削除 ★★★ ---

# 費目全体を抽出するためのデータフレーム
df_main = df[df['details'] == 'all'].copy()

# 主要費目のみに絞り込む（'全体'も除く）
df_main_items = df_main[df_main['expense_items'] != '全体'].copy()

print(f"データロード完了。指定期間: {COMPARISON_YEAR}年 - {LATEST_YEAR}年")
print(f"データフレーム期間: {df_main['period'].min()} - {df_main['period'].max()}")
print(f"データ件数（主要費目のみ）: {len(df_main_items)}")

# 時系列データのインデックス設定
df_ts = df_main_items.set_index(['country', 'period', 'expense_items'])

print("--- データ（抜粋） ---")
display(df_ts.head(), df_ts.tail())

データロード完了。指定期間: 2019年 - 2024年
データフレーム期間: 2019年1-3月期 - 2024年7-9月期
データ件数（主要費目のみ）: 1872
--- データ（抜粋） ---


year period_Quarter Quarter details  \
country period     expense_items                                        
全国籍･地域  2019年1-3月期 宿泊費            2019          1-3月期      1Q     all   
                   飲食費            2019          1-3月期      1Q     all   
                   交通費            2019          1-3月期      1Q     all   
                   娯楽等サービス費       2019          1-3月期      1Q     all   
                   買物代            2019          1-3月期      1Q     all   

                                  consumption_unit  composition_ratio  
country period     expense_items                                       
全国籍･地域  2019年1-3月期 宿泊費                     40848.0               28.2  
                   飲食費                     31023.0               21.4  
                   交通費                     14170.0                9.8  
                   娯楽等サービス費                 6305.0                4.4  
                   買物代                     52352.0               36.2

year period_Quarter Quarter details  \
country period       expense_items                                        
その他     2024年10-12月期 飲食費            2024        10-12月期      4Q     all   
                     交通費            2024        10-12月期      4Q     all   
                     娯楽等サービス費       2024        10-12月期      4Q     all   
                     買物代            2024        10-12月期      4Q     all   
                     その他            2024        10-12月期      4Q     all   

                                    consumption_unit  composition_ratio  
country period       expense_items                                       
その他     2024年10-12月期 飲食費                     83843.0               21.0  
                     交通費                     62787.0               15.7  
                     娯楽等サービス費                13585.0                3.4  
                     買物代                     75520.0               18.9  
                     その他                        47.0                0.0

### 1. 1人あたりの観光消費額の時系列比較による消費単価の変化把握

まず、訪日観光客の**消費単価（1人あたりの消費額）** がコロナ禍前と最新年でどの程度変化したかを確認する。

In [210]:
# 一人あたり消費額データの抽出
df_total_spend = df[(df['expense_items'].str.contains(r'^全体', regex=True, na=False)) & (df['details'] == 'all')].copy()

# 全国籍･地域に絞り込みデータを抽出
df_national = df_total_spend[(df_total_spend['country'] == '全国籍･地域')].copy()

# LATEST_YEARのデータのみを抽出し、year と Quarter でソート
df_latest_national = df_national[df_national['year'] == LATEST_YEAR].sort_values(['year', 'Quarter'])

# 比較対象とする2024年の期間文字列のリストを作成 (ソート済)
latest_quarters = df_latest_national['period'].tolist()

# 3. 比較結果を格納するための空リスト
results = []
is_latest_year_used = not df_latest_national.empty

for latest_period_str in latest_quarters:
    
    # 最新期間に対応する行のデータを取得
    current_row = df_latest_national[df_latest_national['period'] == latest_period_str].iloc[0]
    
    # 比較期間の情報を構築 (period_Quarterの形式を使用)
    comp_quarter_str = current_row['period_Quarter'] 
    comp_period_str = f'{COMPARISON_YEAR}年{comp_quarter_str}'

    # 全国籍･地域 の最新消費単価
    total_spend_latest = current_row['consumption_unit']
    
    # 2019年の比較データを抽出 (df_nationalから)
    df_comp_nat_row = df_national[(df_national['period'] == comp_period_str) & (df_national['country'] == '全国籍･地域')]
    
    if not df_comp_nat_row.empty:
        total_spend_comp = df_comp_nat_row['consumption_unit'].iloc[0]
        
        delta_abs = total_spend_latest - total_spend_comp
        delta_rate = (delta_abs / total_spend_comp) * 100 if total_spend_comp != 0 else np.nan
        
        quarter_only_str = latest_period_str.replace(f'{LATEST_YEAR}年', '')

        # 比較結果に含めるデータ
        results.append({
            '四半期': quarter_only_str,
            #'比較期間 (最新)': latest_period_str,
            #'比較期間 (コロナ前)': comp_period_str,
            f'{COMPARISON_YEAR} 消費単価': total_spend_comp,
            f'{LATEST_YEAR} 消費単価': total_spend_latest,
            '変化額': delta_abs,
            '変化率 (%)': delta_rate,
            'year': current_row['year'],
            'Quarter': current_row['Quarter'] # ソートキーとして使用
        })

# --- (A) 全国籍･地域のサマリテーブル表示 ---
print(f"--- 訪日観光客の一人あたり消費額（消費単価）の変化 ({COMPARISON_YEAR}年 対 {LATEST_YEAR}年) ---")

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(['year', 'Quarter'])

# 表示に使用しないカラムをドロップ
df_display = df_results.drop(columns=['year', 'Quarter'])

# 表示設定
styled_df = df_display.style.format({
    f'{COMPARISON_YEAR} 消費単価': '¥ {:,.0f}', 
    f'{LATEST_YEAR} 消費単価': '¥ {:,.0f}', 
    '変化額': '¥ {:+,.0f}', 
    '変化率 (%)': '{:+.1f} %'
}).hide(axis="index")

display(styled_df)

--- 訪日観光客の一人あたり消費額（消費単価）の変化 (2019年 対 2024年) ---


四半期,2019 消費単価,2024 消費単価,変化額,変化率 (%)
1-3月期,"¥ 144,725","¥ 202,854","¥ +58,129",+40.2 %
4-6月期,"¥ 154,643","¥ 238,389","¥ +83,746",+54.2 %
7-9月期,"¥ 157,134","¥ 215,712","¥ +58,578",+37.3 %
10-12月期,"¥ 166,771","¥ 234,069","¥ +67,298",+40.4 %


訪日観光客の1人あたりの消費額は、2019年の各四半期と比較して2024年では全四半期で大幅に増加していることが確認できる。

### 2. 消費構造の変化

### 2.1 費目別構成比の比較

費目別構成比をコロナ禍前と最新年で比較することにより消費額の増加がどの費目によって牽引されているのかを検討する。

In [211]:
# 比較対象期間のデータ抽出と年平均構成比の計算
df_ratio = df_main_items[df_main_items['country'] == '全国籍･地域'].copy()
df_ratio_yearly = df_ratio.groupby(['year', 'expense_items'])['composition_ratio'].mean().reset_index()

df_latest_ratio = df_ratio_yearly[df_ratio_yearly['year'] == LATEST_YEAR].rename(columns={'composition_ratio': '構成比 (%)'})
df_comp_ratio = df_ratio_yearly[df_ratio_yearly['year'] == COMPARISON_YEAR].rename(columns={'composition_ratio': '構成比 (%)'})

# 2019年 vs 最新年の比較用データフレーム (テーブル表示では使用しないが、変化率計算のために維持)
df_comp_viz = pd.concat([
    df_comp_ratio.assign(比較期間=f'{COMPARISON_YEAR}年平均'),
    df_latest_ratio.assign(比較期間=f'{LATEST_YEAR}年平均')
])

# 変化率の計算とソート
df_merged = pd.merge(
    df_latest_ratio[['expense_items', '構成比 (%)']],
    df_comp_ratio[['expense_items', '構成比 (%)']],
    on='expense_items',
    suffixes=('_latest', '_comp')
)
df_merged['変化率'] = df_merged['構成比 (%)_latest'] - df_merged['構成比 (%)_comp']
df_merged = df_merged.sort_values('変化率', ascending=False)

# 変化率の表 (テーブル表示を強調)
print(f"\n--- 費目別構成比の比較と変化率 ({COMPARISON_YEAR}年 vs {LATEST_YEAR}年, 単位: %) ---")

# カラム名をテーブル表示用にわかりやすく変更
df_table = df_merged[['expense_items', '構成比 (%)_comp', '構成比 (%)_latest', '変化率']].copy()
df_table.columns = ['費目', f'{COMPARISON_YEAR}年 構成比', f'{LATEST_YEAR}年 構成比', '変化率 (増減)']

display(df_table.round(1)) # round(1)で小数点第1位に丸めて表示


--- 費目別構成比の比較と変化率 (2019年 vs 2024年, 単位: %) ---


,費目,2019年 構成比,2024年 構成比,変化率 (増減)
3,宿泊費,28.3,33.5,5.2
2,娯楽等サービス費,3.9,4.5,0.6
1,交通費,10.5,11.0,0.5
5,飲食費,21.3,21.4,0.1
0,その他,0.0,0.0,0.0
4,買物代,36.0,29.7,-6.4


宿泊費の構成比が最も大きく増加し、逆に買物代が最も大きく減少している。
この変化により、総消費単価が増加した中で発生しており、特に **サービス費目（宿泊、娯楽等サービス、交通）の増加**、買物費の相対的な地位低下という、構造的なシフトの可能性が示唆される。

### 2.2 各費目構成比における国籍・地域間の分散構造の比較

各費目構成比の分散構造をコロナ禍前と最新年で比較することにより、国籍・地域の違いによる消費構造の異質性の変化を確認する。

In [212]:
# 国籍・地域ごとのデータ抽出（'全国籍･地域'は除外）
# 元のデータフレーム df_main_items を使用
df_country_data = df_main_items[df_main_items['country'] != '全国籍･地域'].copy()

# 各年・各費目における構成比の標準偏差を計算
df_std = df_country_data.groupby(['year', 'expense_items'])['composition_ratio'].std().reset_index()
df_std = df_std.rename(columns={'composition_ratio': '標準偏差 (STD)'})

# 2019年と最新年の標準偏差を抽出
df_latest_std = df_std[df_std['year'] == LATEST_YEAR].rename(columns={'標準偏差 (STD)': f'{LATEST_YEAR}年 STD'})
df_comp_std = df_std[df_std['year'] == COMPARISON_YEAR].rename(columns={'標準偏差 (STD)': f'{COMPARISON_YEAR}年 STD'})

# 標準偏差の比較と変化量の計算
df_std_merged = pd.merge(
    df_latest_std[['expense_items', f'{LATEST_YEAR}年 STD']],
    df_comp_std[['expense_items', f'{COMPARISON_YEAR}年 STD']],
    on='expense_items'
)

# 標準偏差の変化量 (2024年 - 2019年) を計算
df_std_merged['STD変化量'] = df_std_merged[f'{LATEST_YEAR}年 STD'] - df_std_merged[f'{COMPARISON_YEAR}年 STD']

# 変化量が大きい順（分散構造が拡大した順）にソート
df_std_merged = df_std_merged.sort_values('STD変化量', ascending=False)

# 分散構造の比較結果を表として表示
print(f"\n--- 国籍・地域間の構成比の分散構造比較 ({COMPARISON_YEAR}年 vs {LATEST_YEAR}年) ---")
print("  (STD変化量: +は分散構造が拡大、-は縮小)")

df_std_table = df_std_merged[['expense_items', f'{COMPARISON_YEAR}年 STD', f'{LATEST_YEAR}年 STD', 'STD変化量']].copy()
df_std_table.columns = ['費目', f'{COMPARISON_YEAR}年 STD', f'{LATEST_YEAR}年 STD', 'STD変化量']

# 標準偏差の表を小数点第2位まで丸めて表示
display(df_std_table.round(2))


--- 国籍・地域間の構成比の分散構造比較 (2019年 vs 2024年) ---
  (STD変化量: +は分散構造が拡大、-は縮小)


,費目,2019年 STD,2024年 STD,STD変化量
1,交通費,3.12,3.35,0.23
0,その他,0.06,0.05,-0.01
2,娯楽等サービス費,1.78,1.40,-0.38
3,宿泊費,6.33,5.84,-0.49
5,飲食費,3.02,2.45,-0.56
4,買物代,11.17,9.25,-1.93


各費目構成比の国籍・地域間の分散構造を比較すると、2019年から2024年にかけて、費目ごとに異なる方向の変化が確認される。  

交通費は、唯一分散構造が拡大しており、国籍・地域によって日本国内での移動の度合いに差が生じている可能性がある。  
これは、  
　・一部の国籍では、日本国内を比較的広く移動する滞在スタイル  
　・別の国籍では、特定都市・周辺エリアへの滞在に比重を置くスタイル  
といった **周遊の度合いに関する差異が、国籍・地域間**で拡大している可能性を示唆する。  

一方、宿泊費・飲食費・娯楽等サービス費では分散構造が縮小しており、滞在中のサービス消費の構成比が、国籍・地域を問わず、より近い水準に収束しつつあると解釈できる。  

特に買物代は分散構造の縮小幅が最も大きく、国籍・地域差を特徴づける費目としての相対的重要性が低下している可能性がある。  

以上より、訪日観光消費の構造は、**「何を消費するか」という点よりも、「どの程度日本国内を移動するか」** という側面において、国籍・地域間の違いが相対的に表れやすくなっている可能性が示唆される。

### 3. 費目間相関からみた消費構造の整理

各費目が国籍別消費構造の中でどのように組み合わされて現れているかを確認するため、費目間の相関関係に着目する。

In [213]:
# 比較対象の年
TARGET_YEARS = [2019, 2024]

# データフレームから「全体」を含む費目、および「その他」「不明」を除いた主要費目リストを取得する。
def get_major_expense_items(df_items: pd.DataFrame) -> list:

    excluded_items = ['その他']
    df_filtered_items = df_items[
        ~df_items['expense_items'].str.contains('全体', na=False) &
        ~df_items['expense_items'].isin(excluded_items)
    ]
    
    major_items = df_filtered_items['expense_items'].unique().tolist()
    
    order = ['宿泊費', '飲食費', '買物代', '交通費', '娯楽等サービス費']
    sorted_items = [item for item in order if item in major_items]
    
    for item in major_items:
        if item not in sorted_items:
            sorted_items.append(item)
            
    return sorted_items

# 指定された年と費目について、国籍・地域ごとの四半期平均消費単価を計算し、費目を列とするDataFrameを返す。
def prepare_correlation_data_all_items(df_items: pd.DataFrame, target_years: list, all_expense_items: list) -> pd.DataFrame:

    countries_to_include = [
        c for c in df_items['country'].unique() 
        if c not in ['全国籍･地域', 'その他']
    ]
    
    df_filtered = df_items[
        (df_items['year'].isin(target_years)) & 
        (df_items['expense_items'].isin(all_expense_items)) &
        (df_items['country'].isin(countries_to_include))
    ].copy()
    
    if df_filtered.empty:
        return pd.DataFrame()
        
    df_avg_unit = df_filtered.groupby(['year', 'country', 'expense_items'])['consumption_unit'].mean().reset_index()
    
    df_pivot = df_avg_unit.pivot_table(
        index=['country', 'year'], 
        columns='expense_items', 
        values='consumption_unit'
    ).reset_index()
    
    df_pivot.columns.name = None
    
    return df_pivot

# 相関係数を計算する。
def calculate_correlation(data: pd.DataFrame, x_col: str, y_col: str) -> float:

    if x_col not in data.columns or y_col not in data.columns:
        return np.nan
        
    df_corr_calc = data[[x_col, y_col]].dropna()
    
    if len(df_corr_calc) <= 1:
        return np.nan

    correlation = df_corr_calc[x_col].corr(df_corr_calc[y_col])
    
    return correlation

# 全ての費目ペアについて、年ごとに相関係数を計算し、クロス集計表形式で表示する。
def analyze_all_item_correlations(df_pivot: pd.DataFrame, target_years: list, all_major_items: list):
    
    for year in target_years:
        df_year = df_pivot[df_pivot['year'] == year].copy()
        
        df_corr_matrix = pd.DataFrame(
            index=all_major_items, 
            columns=all_major_items,
            dtype=float
        )
        
        for item_x in all_major_items:
            for item_y in all_major_items:
                if item_x == item_y:
                    df_corr_matrix.loc[item_x, item_y] = 1.0 
                elif pd.isna(df_corr_matrix.loc[item_x, item_y]):
                    correlation = calculate_correlation(df_year, item_x, item_y)
                    df_corr_matrix.loc[item_x, item_y] = correlation
                    df_corr_matrix.loc[item_y, item_x] = correlation
        
        df_corr_matrix.fillna(np.nan, inplace=True)
        
        display(Markdown(f"### -- {year}年 相関行列 --"))
        
        # スタイル設定: ヒートマップ表示と小数点以下3桁表示
        styled_table = (df_corr_matrix.style
            .background_gradient(cmap='RdYlGn', axis=None, vmin=-1, vmax=1)
            .apply(lambda s: ['background-color: lightgray' if s.name == col else '' for col in s.index], axis=1)
            .format("{:.3f}") 
            .highlight_null() # 引数を削除
        )
        
        # クロス集計表（ヒートマップ）の表示
        display(styled_table)
        

        display(Markdown(f"""
            #### {year}年 相関行列の読み取り方

            * **濃い緑色（1.0に近い）**: 非常に強い正の相関。両方の費目で高額を使う国籍・地域が多いことを意味します。
            * **濃い赤色（-1.0に近い）**: 非常に強い負の相関。一方の費目で高額を使う国籍・地域は、他方の費目では低額を使う傾向が強いことを意味します。
            """))

ALL_MAJOR_ITEMS = get_major_expense_items(df_main_items)
    
display(Markdown(f"#### 対象費目リスト: {', '.join(ALL_MAJOR_ITEMS)}"))

df_pivot_data = prepare_correlation_data_all_items(df_main_items, TARGET_YEARS, ALL_MAJOR_ITEMS)

analyze_all_item_correlations(df_pivot_data, TARGET_YEARS, ALL_MAJOR_ITEMS)

#### 対象費目リスト: 宿泊費, 飲食費, 買物代, 交通費, 娯楽等サービス費

### -- 2019年 相関行列 --

,宿泊費,飲食費,買物代,交通費,娯楽等サービス費
宿泊費,1.000,0.971,-0.334,0.962,0.837
飲食費,0.971,1.000,-0.251,0.927,0.803
買物代,-0.334,-0.251,1.000,-0.365,-0.294
交通費,0.962,0.927,-0.365,1.000,0.760
娯楽等サービス費,0.837,0.803,-0.294,0.760,1.000



            #### 2019年 相関行列の読み取り方

            * **濃い緑色（1.0に近い）**: 非常に強い正の相関。両方の費目で高額を使う国籍・地域が多いことを意味します。
            * **濃い赤色（-1.0に近い）**: 非常に強い負の相関。一方の費目で高額を使う国籍・地域は、他方の費目では低額を使う傾向が強いことを意味します。
            

### -- 2024年 相関行列 --

,宿泊費,飲食費,買物代,交通費,娯楽等サービス費
宿泊費,1.000,0.966,-0.201,0.921,0.810
飲食費,0.966,1.000,-0.136,0.910,0.726
買物代,-0.201,-0.136,1.000,-0.303,-0.167
交通費,0.921,0.910,-0.303,1.000,0.675
娯楽等サービス費,0.810,0.726,-0.167,0.675,1.000



            #### 2024年 相関行列の読み取り方

            * **濃い緑色（1.0に近い）**: 非常に強い正の相関。両方の費目で高額を使う国籍・地域が多いことを意味します。
            * **濃い赤色（-1.0に近い）**: 非常に強い負の相関。一方の費目で高額を使う国籍・地域は、他方の費目では低額を使う傾向が強いことを意味します。
            

相関行列を見ると、2019年・2024年のいずれにおいても、宿泊費・飲食費・交通費・娯楽等サービス費の間には高い正の相関が確認される。  
これは、これらのサービス費目が国籍・地域別に見て、**同一の消費構造の中で同時的に高低が現れやすい費目群**であることを示している。

一方で、交通費については、サービス費目群との高い相関を維持しつつも、前節で確認したとおり国籍・地域間の分散構造が拡大している。  
このことから、交通費は**サービス消費構造の一部でありながら、国籍・地域差が相対的に表れやすい費目**として位置づけられる。  

また、買物代は他の費目と負の相関を示し、かつ前節の結果の通り分散構造が大きく縮小している。  
このことから、**サービス消費中心の構造とは異なる特性を持ちつつ、国籍・地域差は縮小傾向にある費目**と整理できる。

### 4. 国内移動を含むサービス消費構造の一般性に関する検証

#### 4.1 費目別消費額に基づく国籍・地域の位置づけ

国内移動（交通費）・宿泊費・娯楽等サービス費・飲食費について、国籍・地域別の消費額水準を確認し、**各費目における上位国が費目間でどの程度重なっているか**を整理する。


In [214]:
TARGET_EXPENSE_ITEMS = ['交通費', '宿泊費', '娯楽等サービス費','飲食費']
# df_main_items は定義済みと仮定

# 指定年の特定の費目（expense_item）の消費単価（四半期平均）を計算し、消費単価の降順にソートしたDataFrameを返す。
def prepare_expense_metrics_unit(df_items: pd.DataFrame, target_year: int, expense_item: str) -> pd.DataFrame:
    all_countries = df_items['country'].unique()
    countries_to_include = [c for c in all_countries if c not in ['全国籍･地域', 'その他']]
    
    df_expense = df_items[
        (df_items['year'] == target_year) & 
        (df_items['expense_items'] == expense_item) & 
        (df_items['country'].isin(countries_to_include))
    ].copy()
    
    df_metrics = df_expense.groupby('country').agg({'consumption_unit': 'mean'}).reset_index()
    df_metrics = df_metrics.sort_values('consumption_unit', ascending=False).reset_index(drop=True)
    return df_metrics

for item in TARGET_EXPENSE_ITEMS:
    # 消費単価と構成比のデータ準備
    df_unit = prepare_expense_metrics_unit(df_main_items, LATEST_YEAR, item)

    # 単価と構成比の表の表示 (元のコードの表示部分を再利用)
    display_expense_unit_table(df_unit, LATEST_YEAR, item)


--- 2024年 交通費 年間消費単価 (四半期平均, 円) (消費単価降順) ---


,国籍・地域,"2024年 交通費年間消費単価 (四半期平均, 円)"
0,スペイン,"71,298"
1,イタリア,"69,304"
2,英国,"57,934"
3,フランス,"57,405"
4,ドイツ,"55,341"
5,オーストラリア,"53,154"
6,カナダ,"47,325"
7,米国,"45,222"
8,ロシア,"44,721"
9,インド,"42,916"



--- 2024年 宿泊費 年間消費単価 (四半期平均, 円) (消費単価降順) ---


,国籍・地域,"2024年 宿泊費年間消費単価 (四半期平均, 円)"
0,英国,"171,845"
1,オーストラリア,"169,842"
2,ドイツ,"159,771"
3,フランス,"150,706"
4,イタリア,"149,080"
5,米国,"144,207"
6,スペイン,"141,047"
7,カナダ,"135,022"
8,ロシア,"124,170"
9,シンガポール,"104,417"



--- 2024年 娯楽等サービス費 年間消費単価 (四半期平均, 円) (消費単価降順) ---


,国籍・地域,"2024年 娯楽等サービス費年間消費単価 (四半期平均, 円)"
0,オーストラリア,"30,168"
1,英国,"22,051"
2,米国,"18,364"
3,スペイン,"16,568"
4,カナダ,"16,262"
5,フランス,"15,771"
6,イタリア,"13,745"
7,ドイツ,"13,295"
8,インド,"11,509"
9,マレーシア,"11,230"



--- 2024年 飲食費 年間消費単価 (四半期平均, 円) (消費単価降順) ---


,国籍・地域,"2024年 飲食費年間消費単価 (四半期平均, 円)"
0,イタリア,"86,298"
1,オーストラリア,"84,566"
2,英国,"84,332"
3,スペイン,"83,287"
4,ドイツ,"81,858"
5,フランス,"75,696"
6,米国,"73,649"
7,カナダ,"73,480"
8,ロシア,"72,273"
9,シンガポール,"66,312"


各費目の消費額水準を見ると、交通費・宿泊費・娯楽等サービス費・飲食費の 上位層 には、  
 　英国、オーストラリア、フランス、ドイツ、イタリア、スペイン、米国、カナダ  
といった国が 複数費目で繰り返し登場しており、上位層に位置する国籍・地域には、一定の重なりが確認される。  

そのため、国内移動を含むサービス費目において高い消費水準を示す国籍・地域は、特定の単一国に限られるものではなく、**複数の国に共通して観測される**可能性が示唆される。

#### 4.2 サービス費目別上位国の重なりに関する定量的確認
前節では、交通費・宿泊費・娯楽等サービス費といったサービス費目において、消費額水準が高い国籍・地域が費目間で一定程度重なっている可能性を確認した。

本節では、この傾向が特定の少数国に限定された現象か、ある程度まとまった国群として観測される構造かを定量的に確認する。そのため、各費目について消費額上位10か国を抽出し、3費目すべてにおいて上位に含まれる国籍・地域の数を確認する。

なお、本節では、前節で確認したサービス費目のうち、国内での移動行動をより直接的に反映すると考えられる交通費・宿泊費・娯楽等サービス費を対象とする。飲食費については、これらの費目との相関が高く、分析に含めた場合に情報の重複が生じる可能性があるため対象から除外する。

In [215]:
# 指定された年と費目について、国籍・地域ごとの消費単価（年平均）を計算し、費目を列とするDataFrameを返す。
def prepare_data_for_ranking(df_items: pd.DataFrame, target_year: int, expense_items: list) -> pd.DataFrame:
    
    # 全ての国籍・地域（全国籍･地域、その他を除く）を特定
    countries_to_include = [
        c for c in df_items['country'].unique() 
        if c not in ['全国籍･地域', 'その他']
    ]
    
    # 対象の年、費目、国籍に絞り込む
    df_filtered = df_items[
        (df_items['year'] == target_year) & 
        (df_items['expense_items'].isin(expense_items)) &
        (df_items['country'].isin(countries_to_include))
    ].copy()
    
    if df_filtered.empty:
        return pd.DataFrame()
        
    # 国籍・地域、費目ごとに消費単価の年平均を計算
    df_avg_unit = df_filtered.groupby(['country', 'expense_items'])['consumption_unit'].mean().reset_index()
    
    # データをピボットし、費目を列にする
    df_pivot = df_avg_unit.pivot_table(
        index='country', 
        columns='expense_items', 
        values='consumption_unit'
    ).reset_index()
    
    df_pivot.columns.name = None
    return df_pivot

# 各費目についてTop Nの国籍・地域を抽出し、その共通集合（重なり数）を計算し、結果のみを表示する。
def analyze_top_n_overlap_summary(df_pivot: pd.DataFrame, service_items: list, top_n: int):
    
    top_n_countries = {}
    
    # 各費目の Top N を抽出
    for item in service_items:
        # 該当費目で降順ソートし、NaNを除外してから Top N を抽出
        df_sorted = df_pivot.sort_values(by=item, ascending=False).dropna(subset=[item])
        
        top_countries = set(df_sorted['country'].head(top_n).tolist())
        top_n_countries[item] = top_countries

    # 共通集合（重なり）の計算
    if not top_n_countries:
        overlap_countries = set()
    else:
        # 3つのセットの共通部分を計算
        all_sets = list(top_n_countries.values())
        overlap_countries = all_sets[0].intersection(*all_sets[1:])
    
    overlap_count = len(overlap_countries)
    
    # 共通上位国籍のデータ表示    
    display(Markdown(f"### 3費目すべてで上位 {TOP_N} に入る国籍・地域"))
    
    # 各費目での順位を計算して表示に追加
    for item in service_items:
        # 元のデータで順位を計算
        df_pivot[f'Rank_{item}'] = df_pivot[item].rank(ascending=False, method='min')
        
    cols_to_show = ['country'] + service_items + [f'Rank_{item}' for item in service_items]
    
    # 重なっている国のみをフィルタリング
    df_overlap_ranked = df_pivot[df_pivot['country'].isin(overlap_countries)][cols_to_show]
    
    # 見やすいように交通費順で並べ替え
    df_overlap_ranked = df_overlap_ranked.sort_values(by=service_items[0], ascending=False).reset_index(drop=True)
    
    # 表示用のフォーマット
    format_dict = {item: "{:,.0f}" for item in service_items}
    format_dict.update({f'Rank_{item}': "{:.0f}位" for item in service_items})
    
    display(df_overlap_ranked.style.format(format_dict))


# データの準備
df_pivot_data = prepare_data_for_ranking(df_main_items, LATEST_YEAR, SERVICE_ITEMS)

# 分析の実行
analyze_top_n_overlap_summary(df_pivot_data, SERVICE_ITEMS, TOP_N)

### 3費目すべてで上位 10 に入る国籍・地域

,country,交通費,宿泊費,娯楽等サービス費,Rank_交通費,Rank_宿泊費,Rank_娯楽等サービス費
0,スペイン,"71,298","141,047","16,568",1位,7位,4位
1,イタリア,"69,304","149,080","13,745",2位,5位,7位
2,英国,"57,934","171,845","22,051",3位,1位,2位
3,フランス,"57,405","150,706","15,771",4位,4位,6位
4,ドイツ,"55,341","159,771","13,295",5位,3位,8位
5,オーストラリア,"53,154","169,842","30,168",6位,2位,1位
6,カナダ,"47,325","135,022","16,262",7位,8位,5位
7,米国,"45,222","144,207","18,364",8位,6位,3位


交通費・宿泊費・娯楽等サービス費の各費目について消費額上位10か国を抽出した結果、3費目すべてにおいて上位に含まれる国籍・地域が8か国確認された。  
これらの国籍・地域は、いずれか単一の費目のみが突出しているのではなく、国内移動を含む複数のサービス費目において、総じて高い消費水準を示している点が共通している。

このことから、国内移動を含むサービス消費が高水準となる構造は、特定の単一国に限定された特殊事例ではなく、**複数の国に共通して観測される消費構造として存在している**可能性が示唆される。

#### 4.3 買物代消費額水準の位置づけ確認
本節では、前節までに確認した国内移動を含むサービス費目の消費構造と対比するため、
買物代について国籍・地域別の消費額水準を確認する。

ここでは、サービス費目において高い消費水準を示した国籍・地域が、
買物代においても同様に上位に位置しているかどうかに着目する。

In [216]:
TARGET_EXPENSE_ITEMS = ['買物代']

for item in TARGET_EXPENSE_ITEMS:
    # データ準備
    df_unit = prepare_expense_metrics_unit(df_main_items, LATEST_YEAR, item)

    # 表の表示
    display_expense_unit_table(df_unit, LATEST_YEAR, item)


--- 2024年 買物代 年間消費単価 (四半期平均, 円) (消費単価降順) ---


,国籍・地域,"2024年 買物代年間消費単価 (四半期平均, 円)"
0,中国,"120,609"
1,香港,"89,293"
2,シンガポール,"76,325"
3,ロシア,"67,411"
4,台湾,"66,780"
5,フィリピン,"66,191"
6,タイ,"62,356"
7,米国,"61,719"
8,フランス,"61,116"
9,ベトナム,"60,394"


買物代の国籍・地域別消費額水準を見ると、中国、香港、シンガポールといった国籍・地域が上位に位置している。
一方で、前節までに確認した、国内移動を含むサービス費目において高い消費水準を示していた国籍・地域は、買物代において必ずしも上位に含まれているわけではない。

こうした違いを踏まえると、交通費・宿泊費・娯楽等サービス費で観測された高水準な消費構造は、モノ消費全般の多寡によって一様に説明されるものではなく、サービス消費に特有の構造である可能性がある。

これらの結果から、訪日観光消費において、**モノ消費を中心とする市場**と、**国内移動を含むサービス消費を中心とする市場**が、必ずしも同一ではない可能性が示唆される。

### 5. 結論とインサイト

本分析では、訪日観光消費について、費目構成比の分散構造、費目間の相関関係、ならびに国籍・地域別の消費額水準を段階的に整理した。
その結果、訪日観光消費の拡大は、来訪者数やモノ消費の多寡のみで説明されるものではなく、**国内移動を含むサービス消費の構造的な違いと強く関係している可能性**が示された。

費目構成比の比較では、宿泊費・飲食費・娯楽等サービス費の国籍差が縮小する一方で、交通費のみ分散構造が拡大しており、日本国内での移動の度合いに関して国籍・地域差が顕在化していることが示唆された。
また、交通費・宿泊費・娯楽等サービス費の間には一貫した正の相関が確認され、**国内移動・滞在・体験型消費が相互に連動した消費構造として成立している可能性**が示された。

さらに、サービス費目別の消費額水準を国籍・地域別に確認した結果、欧米豪を中心とする一定の国群が、交通費・宿泊費・娯楽等サービス費の**すべてで上位に含まれている**ことが確認された。  
このことから、国内移動を含むサービス消費が高水準となる構造は、**特定の単一国に限定された現象ではない**ことが示唆される。  
一方で、買物代の消費額水準はサービス費目とは異なる分布を示しており、訪日観光消費において、モノ消費中心の市場とサービス消費中心の市場が必ずしも一致しない可能性が確認された。

以上より、訪日観光消費の高付加価値化を検討する上では、**「どの国を呼ぶか」という視点に加えて、「来訪者が日本国内でどのように移動し、どのような滞在・体験を重ねるか」という視点が重要である**可能性が示唆される。

今後のインバウンド戦略においては、地域単体の集客に加え、地域間をつなぐ国内動線の設計を意識した施策検討が有効となる可能性がある。